In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
select_features_raw = ['spo2',  'fio2','po2', 'resp_rate',
                  'sodium', 'chloride', 'potassium', 'calcium', 'urineoutput',  'creatinine', 'Scr/bScr', 'bun',
                  'heart_rate','dbp', 'sbp', 'mbp',
                  'wbc', 'temperature',
                  'pco2', 'baseexcess',  'ph',  'aniongap', 'lactate',  'bicarbonate',
                  'glucose','hematocrit', 'hemoglobin', 'platelets', 'bilirubin','inr', 'pt', 'ptt', 'alp', 'ast', 'alt']
df = pd.read_csv("../00.data_merge/df_saki_timeseries_feature_all.csv")
df = df.rename(columns={"crea_divide_basecrea":"Scr/bScr"})   

fix_feature = ["dataset","groupHPD","time","stay_id"]
select_features = select_features_raw + fix_feature

# Time Selection
time_lst = [-2,-1,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28]
df = df[df["time"].isin(time_lst)]
time_dict = {-2:-1,-1:-1,1:1,2:1,3:1,4:1,5:2,6:2,7:2,8:2,9:3,10:3,11:3,12:3,13:4,14:4,15:4,16:4,17:5,18:5,19:5,20:5,21:6,22:6,23:6,24:6,25:7,26:7,27:7,28:7}
df["time"] = df["time"].map(time_dict)

df_fea_add_1 = df.groupby(["stay_id","time"]).agg("mean").reset_index()
df_fea_add_1 = df_fea_add_1.drop(["urineoutput"],axis=1)
df_fea_add_2 = df.groupby(["stay_id","time"]).agg({"urineoutput":"sum"}).reset_index()
df_fea_add = pd.merge(df_fea_add_1,df_fea_add_2,how="inner",on=["stay_id","time"])

<ipython-input-1-89d80c1e65f1>:21: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_fea_add_1 = df.groupby(["stay_id","time"]).agg("mean").reset_index()


In [2]:
df_grouped = df.groupby(["dataset","groupHPD","time"]).agg('mean').reset_index()
df_grouped = df_grouped[select_features]

dataset_dict = {"aumcdb":2, "mimic":1,"eicu":3}
df_grouped["dataset"] = df_grouped["dataset"].map(dataset_dict)
df_grouped = df_grouped.sort_values(["groupHPD","dataset","time"])
df_grouped = df_grouped.drop(["stay_id"],axis=1)
df_grouped["groupHPD"] = df_grouped["groupHPD"].map({1:"C1", 2:"C2", 3:"C3"})
df_grouped[select_features_raw] = df_grouped[select_features_raw].apply(lambda x: round(x, 4))

df_grouped_cov = df_grouped.melt(["dataset","groupHPD","time"])
df_grouped_cov = df_grouped_cov[~df_grouped_cov.value.isna()]

In [3]:
map_dict = {'temperature':'Temperature', 'mbp':'MBP', 'dbp':'DBP', 'sbp':'SBP', 'ph':'pH',
       'bicarbonate':'Bicarbonate', 'baseexcess':'Base Excess', 'pco2':'Pco2', 'calcium':'Calcium', 'sodium': 'Sodium',
       'creatinine':'Creatinine', 'potassium':'Potassium', 'bilirubin':'Bilirubin', 'resp_rate':'Respiratory Rate',
       'heart_rate':'Heart Rate', 'aniongap':'Aniongap', 'lactate': 'Lactate', 'wbc':'WBC', 'glucose':'Glucose', 'fio2':'Fio2', 'po2':'Po2',
       'spo2':'Spo2', 'chloride':'Chloride', 'hemoglobin':'Hemoglobin', 'hematocrit':'Hematocrit',"Scr/bScr":"Scr/bScr",
       'inr':'Inr', 'pt':'PT', 'ptt':'PTT', 'alp':'Alp', 'ast':'Ast', 'alt':'Alt','bun':'BUN',"urineoutput":"Urine Output",
        'platelets':'Platelets'}
df_grouped_cov["variable"] = df_grouped_cov["variable"].map(map_dict)
print(df_grouped_cov["variable"].unique())

['Spo2' 'Fio2' 'Po2' 'Respiratory Rate' 'Sodium' 'Chloride' 'Potassium'
 'Calcium' 'Urine Output' 'Creatinine' 'Scr/bScr' 'BUN' 'Heart Rate' 'DBP'
 'SBP' 'MBP' 'WBC' 'Temperature' 'Pco2' 'Base Excess' 'pH' 'Aniongap'
 'Lactate' 'Bicarbonate' 'Glucose' 'Hematocrit' 'Hemoglobin' 'Platelets'
 'Bilirubin' 'Inr' 'PT' 'PTT' 'Alp' 'Ast' 'Alt']


In [4]:
#MIMIC
df_data = df_grouped_cov[df_grouped_cov["dataset"]==1]
print(df_data.variable.unique())
df_data = df_data.pivot_table(index="variable",columns=["groupHPD","time"],values="value").reset_index()
#df_data = df_data[df_data['variable'].isin(select_features)].sort_values(by=['variable'], key=lambda x: x.map({v: i for i, v in enumerate(select_features)}))
df_data.to_csv("df_mimic_other_feature.csv",index=False)

# AUMCdb
df_data = df_grouped_cov[df_grouped_cov["dataset"]==2]
print(df_data.variable.unique())
df_data = df_data.pivot(index="variable",columns=["groupHPD","time"],values="value").reset_index()
#df_data = df_data[df_data['variable'].isin(select_features)].sort_values(by=['variable'], key=lambda x: x.map({v: i for i, v in enumerate(select_features)}))
df_data.to_csv("df_aumcdb_other_feature.csv",index=False)

# eICU
df_data = df_grouped_cov[df_grouped_cov["dataset"]==3]
print(df_data.variable.unique())

df_data = df_data.pivot(index="variable",columns=["groupHPD","time"],values="value").reset_index()
#df_data = df_data[df_data['variable'].isin(select_features)].sort_values(by=['variable'], key=lambda x: x.map({v: i for i, v in enumerate(select_features)}))
df_data.to_csv("df_eicu_other_feature.csv",index=False)

['Spo2' 'Fio2' 'Po2' 'Respiratory Rate' 'Sodium' 'Chloride' 'Potassium'
 'Calcium' 'Urine Output' 'Creatinine' 'Scr/bScr' 'BUN' 'Heart Rate' 'DBP'
 'SBP' 'MBP' 'WBC' 'Temperature' 'Pco2' 'Base Excess' 'pH' 'Aniongap'
 'Lactate' 'Bicarbonate' 'Glucose' 'Hematocrit' 'Hemoglobin' 'Platelets'
 'Bilirubin' 'Inr' 'PT' 'PTT' 'Alp' 'Ast' 'Alt']
['Spo2' 'Fio2' 'Po2' 'Respiratory Rate' 'Sodium' 'Chloride' 'Potassium'
 'Calcium' 'Urine Output' 'Creatinine' 'Scr/bScr' 'Heart Rate' 'DBP' 'SBP'
 'MBP' 'WBC' 'Temperature' 'Pco2' 'Base Excess' 'pH' 'Aniongap' 'Lactate'
 'Bicarbonate' 'Glucose' 'Hematocrit' 'Hemoglobin' 'Bilirubin' 'PT' 'PTT']
['Spo2' 'Fio2' 'Po2' 'Respiratory Rate' 'Sodium' 'Chloride' 'Potassium'
 'Calcium' 'Urine Output' 'Creatinine' 'Scr/bScr' 'BUN' 'Heart Rate' 'DBP'
 'SBP' 'MBP' 'WBC' 'Temperature' 'Pco2' 'Base Excess' 'pH' 'Aniongap'
 'Lactate' 'Bicarbonate' 'Glucose' 'Hematocrit' 'Hemoglobin' 'Platelets'
 'Bilirubin' 'Inr' 'Alp' 'Ast' 'Alt']


In [5]:
df_data

groupHPD          variable        C1                                          \
time                              -1         1         2         3         4   
0                      Alp  115.4935  111.7057  106.7703  108.2058  111.1594   
1                      Alt   53.6885   54.7063   58.1466   56.7205   55.5619   
2                 Aniongap   10.9178   10.3163    9.7890    9.4753    9.0590   
3                      Ast   62.9580   62.6404   65.3760   59.7145   55.0700   
4                      BUN   28.7064   30.8359   31.9773   32.0037   32.2584   
5              Base Excess   -0.8124   -0.9173   -0.5864   -0.2003    0.4672   
6              Bicarbonate   24.4745   24.4846   24.9699   25.5682   26.2327   
7                Bilirubin    1.2304    1.1608    1.1449    1.1112    1.1450   
8                  Calcium    8.3701    8.2059    8.2400    8.3286    8.3731   
9                 Chloride  104.0942  105.2381  105.8736  105.8679  105.9381   
10              Creatinine    1.1422    1.1987    1.1654    1.1166    1.0929   
11                     DBP   61.8208   61.6265   62.2962   62.1529   62.7857   
12                    Fio2    0.5491    0.5189    0.4957    0.4879    0.4838   
13                 Glucose  155.8308  146.1947  145.5299  145.9385  146.3388   
14              Heart Rate   92.8730   88.3836   87.3407   87.3108   86.5393   
15              Hematocrit   33.6206   32.2692   31.3333   31.2426   31.4278   
16              Hemoglobin   11.0136   10.5510   10.2532   10.2087   10.2587   
17                     Inr    1.6222    1.6400    1.5838    1.4648    1.4018   
18                 Lactate    2.3255    2.1136    1.9888    1.8423    1.8135   
19                     MBP   78.3553   78.5178   78.8686   79.2299   80.0650   
20                    Pco2   41.0429   40.9176   40.9025   41.2066   41.5571   
21               Platelets  212.8048  200.6329  192.4436  192.4704  198.2661   
22                     Po2  100.7643  100.4980   97.5450   95.1985   94.2488   
23               Potassium    4.1227    4.0786    3.9902    3.9446    3.9465   
24        Respiratory Rate   21.7379   20.9718   20.9198   20.8091   20.7158   
25                     SBP  114.4648  116.1346  116.4275  116.9583  117.8395   
26                Scr/bScr    1.1413    1.2011    1.1777    1.1277    1.0999   
27                  Sodium  138.6787  139.2095  139.7740  140.1727  140.5335   
28                    Spo2   95.8444   96.0581   95.8358   95.7362   95.7151   
29             Temperature   37.0037   36.8848   36.8261   36.8244   36.8149   
30            Urine Output  105.9751  106.0373  101.2995   87.1654   77.7521   
31                     WBC   14.1770   13.8594   13.3612   12.5342   12.3285   
32                      pH    7.3824    7.3816    7.3863    7.3906    7.3985   

groupHPD                                      C2  ...                      \
time             5         6         7        -1  ...         6         7   
0         112.7755  117.6560  122.0875  111.5926  ...  109.2480  107.7859   
1          54.7370   49.6969   49.2459   46.4466  ...   34.8110   36.5448   
2           9.0385    9.0594    8.8368   10.4588  ...    8.6884    8.7327   
3          54.1354   49.6876   48.7085   54.3881  ...   35.7016   37.4421   
4          32.9110   33.2342   34.3711   17.4206  ...   15.8722   16.0238   
5           0.7101    1.1313    1.6107   -0.6936  ...    1.4001    1.6175   
6          26.5259   27.0322   27.2691   24.8665  ...   27.7239   27.8612   
7           1.1381    1.1334    1.1177    1.0996  ...    0.9338    0.9093   
8           8.4081    8.4022    8.3545    8.3535  ...    8.3267    8.3712   
9         105.7561  105.5487  105.5982  103.5736  ...  103.8858  103.7316   
10          1.0601    1.0429    1.0353    0.8605  ...    0.6993    0.6990   
11         62.8094   62.2464   62.3160   62.5260  ...   64.0213   64.7403   
12          0.4786    0.4837    0.4785    0.5130  ...    0.4556    0.4594   
13        147.6772  148.0230  147.6811  144.